# Image Feature Extraction

This notebook applies augmentations to images and extracts features (histograms + HOG) to save in a CSV file.

**Dependencies:** numpy, pandas, Pillow, scikit-image, matplotlib, tqdm

In [16]:
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
from skimage.feature import hog
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from tqdm import tqdm

ROOT = Path('.')
IMAGES_DIR = ROOT / 'images'
OUTPUT_CSV = ROOT / 'image_features.csv'

In [17]:
def get_image_files(images_dir):
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(list(images_dir.glob(ext)))
    return sorted(image_files)

FIXED_SIZE = (128, 128)

def apply_augmentations(image):
    augmentations = []

    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    image = image.resize(FIXED_SIZE, Image.Resampling.LANCZOS)
    
    augmentations.append((image.copy(), 'original'))
    
    augmentations.append((image.rotate(90, expand=False), 'rot90'))
    
    augmentations.append((ImageOps.mirror(image), 'flip_h'))
    
    gray_img = image.convert('L').convert('RGB')
    augmentations.append((gray_img, 'grayscale'))
    
    return augmentations

def extract_color_histogram(image, bins=16):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    img_array = np.array(image)
    hist_features = []
    
    for channel in range(3):
        hist, _ = np.histogram(img_array[:,:,channel], bins=bins, range=(0, 255), density=True)
        hist_features.extend(hist)
    
    return np.array(hist_features)

def extract_hog_features(image):
    if image.mode == 'RGB':
        gray_array = rgb2gray(np.array(image))
    else:
        gray_array = np.array(image.convert('L')) / 255.0
    
    features = hog(
        gray_array,
        orientations=9,
        pixels_per_cell=(16, 16),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        feature_vector=True
    )
    
    return features

print("Feature extraction functions defined.")
print(f"Fixed image size: {FIXED_SIZE}")
print(f"Expected color histogram features: {16 * 3}")
print(f"Expected HOG features: {((128//16-1)//2) * ((128//16-1)//2) * 9}")

Feature extraction functions defined.
Fixed image size: (128, 128)
Expected color histogram features: 48
Expected HOG features: 81


In [18]:
def process_all_images():
    image_files = get_image_files(IMAGES_DIR)
    print(f"Found {len(image_files)} images to process")
    
    all_data = []
    expected_feature_count = None
    
    for img_path in tqdm(image_files, desc="Processing images"):
        try:
            image = Image.open(img_path)
            filename = img_path.name
            
            augmentations = apply_augmentations(image)
            
            for aug_image, aug_type in augmentations:
                assert aug_image.size == FIXED_SIZE, f"Image size mismatch: {aug_image.size} != {FIXED_SIZE}"
                
                color_hist = extract_color_histogram(aug_image)
                hog_feats = extract_hog_features(aug_image)
                
                combined_features = np.concatenate([color_hist, hog_feats])
                
                if expected_feature_count is None:
                    expected_feature_count = len(combined_features)
                    print(f"Feature vector length: {expected_feature_count}")
                else:
                    assert len(combined_features) == expected_feature_count, f"Feature count mismatch: {len(combined_features)} != {expected_feature_count}"
                
                row_data = {
                    'filename': filename,
                    'augmentation': aug_type,
                    'width': aug_image.width,
                    'height': aug_image.height
                }

                for i, feat_val in enumerate(combined_features):
                    row_data[f'feature_{i}'] = feat_val
                
                all_data.append(row_data)
                
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue
    
    df = pd.DataFrame(all_data)
    
    feature_cols = [col for col in df.columns if col.startswith('feature_')]
    nan_count = df[feature_cols].isna().sum().sum()
    print(f"NaN values in features: {nan_count}")
    
    df.to_csv(OUTPUT_CSV, index=False)
    
    print(f"Processing complete!")
    print(f"Total rows: {len(df)}")
    print(f"Features per image: {len(feature_cols)}")
    print(f"CSV saved to: {OUTPUT_CSV}")
    
    return df

df_results = process_all_images()

Found 11 images to process


Processing images:   0%|          | 0/11 [00:00<?, ?it/s]

Processing images:   9%|▉         | 1/11 [00:00<00:01,  9.27it/s]

Feature vector length: 1812


Processing images: 100%|██████████| 11/11 [00:01<00:00,  9.38it/s]



NaN values in features: 0
Processing complete!
Total rows: 44
Features per image: 1812
CSV saved to: image_features.csv
Processing complete!
Total rows: 44
Features per image: 1812
CSV saved to: image_features.csv


In [19]:
print("DataFrame Info:")
print(f"Shape: {df_results.shape}")
print(f"Columns: {list(df_results.columns[:10])}...")  # Show first 10 columns

print("\nFirst few rows:")
display(df_results.head())

print("\nAugmentation counts:")
print(df_results['augmentation'].value_counts())

if OUTPUT_CSV.exists():
    print(f"\nCSV file successfully created: {OUTPUT_CSV}")
    print(f"File size: {OUTPUT_CSV.stat().st_size} bytes")
else:
    print(f"\nCSV file was not created!")

print(f"\nProcessing complete!")

DataFrame Info:
Shape: (44, 1816)
Columns: ['filename', 'augmentation', 'width', 'height', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5']...

First few rows:


,filename,augmentation,width,height,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_1802,feature_1803,feature_1804,feature_1805,feature_1806,feature_1807,feature_1808,feature_1809,feature_1810,feature_1811
0,Best-neutral.jpg,original,128,128,0.006051,0.006074,0.003952,0.002876,0.003037,0.003512,...,0.000690,0.043561,0.126216,0.109087,0.143499,0.344723,0.215184,0.039635,0.034399,0.010474
1,Best-neutral.jpg,rot90,128,128,0.006051,0.006074,0.003952,0.002876,0.003037,0.003512,...,0.328049,0.063561,0.038447,0.017445,0.023641,0.033754,0.032575,0.161954,0.318871,0.086670
2,Best-neutral.jpg,flip_h,128,128,0.006051,0.006074,0.003952,0.002876,0.003037,0.003512,...,0.058132,0.044297,0.019638,0.032914,0.018356,0.079153,0.229773,0.285169,0.072223,0.005691
3,Best-neutral.jpg,grayscale,128,128,0.006399,0.006350,0.004485,0.004193,0.003795,0.003807,...,0.000000,0.044469,0.127087,0.109996,0.143645,0.347410,0.212056,0.041094,0.032934,0.009455
4,Best-smiling.jpg,original,128,128,0.006679,0.005947,0.003791,0.002777,0.003600,0.003244,...,0.005482,0.056361,0.146068,0.099460,0.103971,0.312100,0.258654,0.025312,0.026713,0.013290



Augmentation counts:
augmentation
original     11
rot90        11
flip_h       11
grayscale    11
Name: count, dtype: int64

CSV file successfully created: image_features.csv
File size: 1580600 bytes

Processing complete!
